# このファイルについて

衛星の熱設計における熱平衡方程式を解きます。
\begin{align*}
 m_ic_i\frac{dT_i}{dt}  = &\sum c_{ij}(T_i-T_j) + \sum \sigma R_{ij} (T_j^4 - T_i^4)\\
 & + (太陽輻射・アルベド)_i+ (地球からの赤外輻射)_i + (内部機器の発熱)_i
\end{align*}

アルベド項と赤外輻射項が時間変動するので厳しい。。。

頑張って

# Requirement

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

# 関数など

In [2]:
class Thermal_simu():
    """熱平衡方程式をとくクラス。"""
    def __init__(self,capacity_i,C_ij,R_ij,r_satellite):
        """
        初期化
        Parameters:
        capacity_i: ndarray 1-D vector
            節点iでの熱容量
        C_ij: ndarray 2-D vector
            節点i,jの伝熱によるカップリング
        R_ij: ndarray 2-D vector
            節点i,jの輻射によるカップリング
        r_satellite: ndarray 2-D vector
            衛星の軌道半径。太陽輻射とか計算するときに使う。と思う。
        """
        c_i = self.capacity_i
        C_ij = self.C_ij
        R_ij = self.R_ij
        r = r_satellite
        
        P_s = 1358 # 太陽定数
        
        T_i = np.zeros(i) # 最終的に、熱平衡方程式を解いた結果をここに格納したい
    
    def solve_eq(self):
        """
        全部解いてくれる関数だよ。
        """
        return 0
        
    def sun_radiation(self):
        """
        太陽輻射 

        Parameters:

        Returns:
        q: ndarray
            i番目の成分が節点iにおける太陽輻射による熱入力に対応
        """
        q = 0
        return q
    
    def earth_radiation(self):
        """
        地球の赤外輻射

        Parameters:

        Returns:
        q: ndarray
            i番目の成分が節点iにおける地球の赤外輻射による熱入力に対応
        """
        q = 0
        return q
    
    def albedo(self):
        """
        地球のアルベド

        Parameters:

        Returns:
        q: ndarray
            i番目の成分が節点iにおける地球のアルベドによる熱入力に対応
        """
        q = 0
        return q

In [3]:
class Easy_Thermal_simu():
    """熱平衡方程式をとくクラス。"""
    def __init__(self,A_i,alpha_i,eps_i):
        self.A = A_i
        self.alpha = alpha_i
        self.eps = eps_i
        self.P_s = 1358 # 太陽定数
        self.P_e = 237
        self.T = 0 # 最終的に、熱平衡方程式を解いた結果をここに格納したい
        self.Q_inner = 0
        self.sigma = 5.67*10**(-8)
    
    def solve_eq(self):
        """
        全部解いてくれる関数だよ。
        """
        Q_out_by_T4 = 0
        for i in range(len(self.A)):
            Q_out_by_T4 += self.A[i] * self.eps[i] * self.sigma
        Q_in = self.get_sun_radiation() + self.get_earth_radiation() + self.get_albedo() + self.Q_inner
        self.T = (Q_in/Q_out_by_T4)**0.25
        
    def get_sun_radiation(self):
        """
        太陽輻射 

        Parameters:

        Returns:
        q: ndarray
            i番目の成分が節点iにおける太陽輻射による熱入力に対応
        """
        q = self.P_s * self.A[0] * self.alpha[0] * 1
        return q
    
    def get_earth_radiation(self):
        """
        地球の赤外輻射

        Parameters:

        Returns:
        q: ndarray
            i番目の成分が節点iにおける地球の赤外輻射による熱入力に対応
        """
        q = self.P_e * self.A[5] * self.eps[5] * 1 
        return q
    
    def get_albedo(self):
        """
        地球のアルベド

        Parameters:

        Returns:
        q: ndarray
            i番目の成分が節点iにおける地球のアルベドによる熱入力に対応
        """
        q = self.P_s * 0.3 * self.A[5] * self.alpha[5] * 1
        return q
    
    def set_Q_inner(self,q):
        self.Q_inner = q

In [4]:
alpha = [0.2,0.2,0.2,0.2,0.2,0.2]
eps = [0.8,0.8,0.8,0.8,0.8,0.8]
A = [1,1,1,1,1,1]

sim = Easy_Thermal_simu(A,alpha,eps)
sim.set_Q_inner(500)
sim.solve_eq()
sim.T

248.7892934630962

\begin{align*}
 m_ic_i\frac{dT_i}{dt}  = &\sum c_{ij}(T_i-T_j) + \sum \sigma R_{ij} (T_j^4 - T_i^4)\\
 & + (太陽輻射・アルベド)_i+ (地球からの赤外輻射)_i + (内部機器の発熱)_i
\end{align*}

In [36]:
n_nodes = 2 # 節点数
n_steps = 100
dt = 1

sigma = 1

t = np.arange(0,dt*n_steps,n_steps)              # 時間
m = np.ones(n_nodes)              # 質量
C = np.ones(n_nodes)              # 熱容量
c = np.ones([n_nodes,n_nodes])    # c[i,j]
R = np.ones([n_nodes,n_nodes])    # R[i,j]
T_all = np.ones([n_nodes,n_steps])    # 温度（横に時間軸）

In [46]:
def get_dTdt(T):
    dTdt = [0 for i in range(n_nodes)]
    for i in range(n_nodes):
        sum_1 = sum([c[i][j]*(T[i]-T[j]) for j in range(n_nodes)])
        sum_2 = sigma * sum([R[i][j]*(T[j]**4-T[i]**4) for j in range(n_nodes)])
        dTdt[i] = (sum_1 + sum_2) / m[i] / C[i]
    return dTdt

In [47]:
def solve

[14.0, -14.0]